Dans les [manipulations d'un tableau](https://docs.scipy.org/doc/numpy-1.15.1/reference/routines.array-manipulation.html) on comprend :

* la réorganisation (réindexation) du tableau
* l'aggrégation de 2 tableaux ou plus
* le découpage d'un tableau en 2 ou plus

mais avant de regarder ces points, regardons comment Numpy présente les
dimensions d'un tableau multidimensionel avec la notion d'axes.

## Les axes

Un tableau a des axes qui correspondent aux axes d'un repère dans l'espace. L'ordre des axes
est celui de l'inclusion des crochets. En 2D un tableau de tableau est un tableau de lignes avec
chaque ligne qui est un tableau 1D de valeurs. L'ordre est donc lignes puis colonnes (contrairement à l'axes $(x,y)$ dans
l'espace).

De très nombreuses opérations sur les tableaux se font suivant un des axes du tableau aussi il est important de 
comprendre ce que sont les axes.

Regardons sur un exemple

In [1]:
import numpy as np

a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

Faire la sommes des valeurs suivant l'axe 1 revient à additionner ce qui varie suivant l'axe 1 donc dans la direction des colonnes (à savoir le contenu de chaque ligne). L'axe 0 serait suivant les lignes (donc la somme du contenu de chaque colonne).

In [2]:
a.sum(axis=1)   # 0+1+2 = 3 and 3+4+5 = 12

array([ 3, 12])

Un autre facon de voir les axes est de les considérer comme des __axes de projection__. Si je projette un objet 3D
suivant l'axe des $x$, le résultat est un objet 2D en $(y,z)$.

Si je somme sur l'axe 0 un tableau de dimension (2,3,4) cela veut dire que je perds la dimension 0 et donc la dimension du résultat est (3,4). Pour chaque case
de ce tableau résultat on a additionné 2 valeurs puisque la dimension 0 du tableau était 2.

In [3]:
a = np.arange(24).reshape(2,3,4)
a.prod(axis=0)

array([[  0,  13,  28,  45],
       [ 64,  85, 108, 133],
       [160, 189, 220, 253]])

#### Quelques fonctions qui supportent les axes

Toutes les fonctions qui s'appliquent à un ensemble de valeur pour produire un résultat
doivent pouvoir utiliser de concept d'axe (je ne les ai pas toutes
vérifiées mais n'hésitez pas à m'indiquer un contre-exemple). On a les fonctions mathémaiques suivantes :

* arithmétiques : `sum`, `prod`, `cumsum`, `cumprod` 
* statistiques : `max`, `min`, `mean` (moyenne), `average` (moyenne pondérée), `std` (écart type), `var`, `median`, `percentile`, `quantile`
* autres : `gradiant`, `diff`, `fft`

De plus il est possible de trier les valeurs d'un tableau suivant l'axe de son choix avec `sort`.
Par contre on ne peut les mélanger, avec [`shuffle`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.shuffle.html#numpy.random.shuffle), que suivant l'axe 0.

#### Appliquer sa fonction suivant un axe

La fonction [`apply_along_axis`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.apply_along_axis.html)
permet d'appliquer la fonction de son choix à un tableau suivant un axe :

In [4]:
def last_minus_first(a):
    return a[-1] - a[0]

a= np.array([[5,4,2],[3,1,7]])
np.apply_along_axis(last_minus_first, axis=1, arr=a)

array([-3,  4])

#### Appliquer une fonction suivant plusieurs axes

Certaines opérations peuvent prendre une liste d'axes et non un seul axe. 

Il est également peut utiliser la fonction [`apply_over_axes`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.apply_over_axes.html#numpy.apply_over_axes) pour lui indiquer quelle
fonction doit être appliquées suivant les axes donnés.

In [5]:
a = np.arange(24).reshape(2,3,4)
print('a.max \n', a.max(axis=(0,1)), '\n') 
print('a.max keepdim \n', a.max(axis=(0,1), keepdims=True), '\n') 
print('apply max \n', np.apply_over_axes(np.max, a, axes=(0,1)), '\n')

a.max 
 [20 21 22 23] 

a.max keepdim 
 [[[20 21 22 23]]] 

apply max 
 [[[20 21 22 23]]] 



In [6]:
np.apply_over_axes(np.max, a, axes=(1,))

array([[[ 8,  9, 10, 11]],

       [[20, 21, 22, 23]]])

## Réorganisation d'un tableau

On a déjà vu `reshape` pour changer la forme d'un tableau, `flatten` pour l'applatir en 1 dimension, regardons
d'autres fonctions de manipulation des tableaux.

#### `moveaxis` déplace un axe

Imaginons que j'ai deux classes d'élèves avec pour chaque classe les notes de 4 élèves dans trois matières. Pour
qu'on visualise bien, la première classe n'a que des note impaire et la seconde que des notes paires. Quand aux
matières les élèves ont tous en dessous de 10 pour la première et au dessus de 10 pour la 3e.

La fonction `moveaxis` permet de déplacer un axe. Si ainsi je désire que les matières deviennent le premier axe
afin d'en faire ressortir les notes, je déplace l'axe 2 à la position 0, les axes 0 et 1 devenant les axes 1 et 2 :

In [7]:
a = np.array([[[3,11,17],[7,7,15],[5,9,11],[1,17,15]],    # grades of the 4 students of class #1 for each topic
              [[6,12,14],[8,10,12],[6,12,16],[2,6,12]]])  # grades of the 4 students of class #2 for each topic
print('a.shape = ',a.shape, '\n')
print(a, '\n')
b = np.moveaxis(a,2,0) 
print('b.shape = ',b.shape)
b

a.shape =  (2, 4, 3) 

[[[ 3 11 17]
  [ 7  7 15]
  [ 5  9 11]
  [ 1 17 15]]

 [[ 6 12 14]
  [ 8 10 12]
  [ 6 12 16]
  [ 2  6 12]]] 

b.shape =  (3, 2, 4)


array([[[ 3,  7,  5,  1],
        [ 6,  8,  6,  2]],

       [[11,  7,  9, 17],
        [12, 10, 12,  6]],

       [[17, 15, 11, 15],
        [14, 12, 16, 12]]])

#### `swapaxes` échange 2 axes

Plutôt que d'insérer un axe à une nouvelle position et faire glisser les autres, on peut vouloir en échanger deux.
Voici comme avoir les notes pour chaque classe et pour chaque matière : 

In [8]:
a.swapaxes(1,2)

array([[[ 3,  7,  5,  1],
        [11,  7,  9, 17],
        [17, 15, 11, 15]],

       [[ 6,  8,  6,  2],
        [12, 10, 12,  6],
        [14, 12, 16, 12]]])

#### Changer l'ordre des éléments d'un tableau

On peut inverser les valeurs d'un tableau suivant un axe avec [`flip`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.flip.html#numpy.flip)
ce qui peut aussi être fait en l'indiquant au niveau des indices.  Ainsi `np.flip(a, n)` est équivalent à 
`a[:,:,..,::-1,:,...,:]` avec `::-1` en $n$-ième position.

On peut faire glisser les valeurs suivant un axe avec `roll` en spécifiant de combien on les fait glisser :

In [9]:
a = np.arange(8).reshape([2,4])
np.roll(a, 2, axis=1)              # roll elements by 2 along axis 1

array([[2, 3, 0, 1],
       [6, 7, 4, 5]])

## Aggrégation

#### Concaténation

La fonction de base est [`concatenate`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.concatenate.html) en indiquant l'axe choisi pour la concaténation. C'est à mon avis la méthode
la plus sûre et elle marche quelque soit la dimension.

Cela étant on peut utiliser pour des tableaux 2D ou 3D :

* `vstack` ou `row_stack` pour la concaténation vertical 
* `hstack` ou `column_stack` pour la concaténation horizontal
* `dstack` pour la concaténation en profondeur (_deep_).

Toutes ces fonctions prennent une liste de tableaux à concaténer comme argument. Bien sûr les tailles des tableaux doivent être compatibles.

In [10]:
a = np.arange(6).reshape(2,3)
b = np.ones((2,3))

print(np.concatenate((a,b), axis=0), '\n')   # same than vstack
print(np.hstack((a,b)))                      # same than concatenate with axis=1

[[0. 1. 2.]
 [3. 4. 5.]
 [1. 1. 1.]
 [1. 1. 1.]] 

[[0. 1. 2. 1. 1. 1.]
 [3. 4. 5. 1. 1. 1.]]


#### Empilage

A la différence de la concaténation, l'empilage ajoute une dimension.
Empiler est utile pour stocker un paquet de tableaux 2D, des images par exemple, dans un tableau 3D. 
On utilise la fonction [`stack`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.stack.html).

In [11]:
a = np.arange(6).reshape(2,3)
b = np.ones((2,3))

c = np.stack((a,b))   #  c[0] is a
c

array([[[0., 1., 2.],
        [3., 4., 5.]],

       [[1., 1., 1.],
        [1., 1., 1.]]])

Notez que `stack` a une option `axis` pour indiquer la direction dans laquelle on désire stocker les tableaux données.

## Découpage

Après la concaténation, le découpage avec [`split`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.split.html#numpy.split) qui demande comme arguments :

* le tableau à découper
* en combien de morceaux ou à quels indices
* la direction (l'axe)

Pour retrouver nos deux tableaux qui ont généré le résultat de la cellule précédante on coupe en 2 suivant l'axe 0 :

In [12]:
e,f = np.split(c,2,0)
print(e, '\n')
print(f)

[[[0. 1. 2.]
  [3. 4. 5.]]] 

[[[1. 1. 1.]
  [1. 1. 1.]]]


Il existe aussi `hsplit`, `vsplit` et `dsplit` pour découper suivant les axes 0, 1 et 2.

## Pandas

On retrouvera ces manipulations avec Pandas qui est le super tableur de Python et donc qui travaille aussi sur des 
structures en forme de tableau mais sans la contrainte que toutes les valeurs soient du même type.

{{ PreviousNext("np02 Filtres.ipynb", "np04 Calcul matriciel.ipynb")}}